# **Sample EDA**

In [42]:
import pandas as pd
from pandasql import sqldf
import plotly.express as px
import plotly.graph_objects as go

### **Step 1: Preview Dataset**

In [18]:
training_data = pd.read_parquet('clean_car_listings.parquet')
training_data

,price,model_year,make,model,trim,mileage,exterior_color,interior_color,num_accidents,num_owners,usage_type,city,state
0,13895,2006,BMW,Z4,Roadster 3.0si,114889,White,Unknown,0,5,Personal,Tempe,AZ
1,19888,2008,BMW,M5,Sedan,129195,Blue,Black,0,3,Personal,Tempe,AZ
2,19999,2008,BMW,M6,Coupe,93700,Gray,Black,0,2,Fleet,West Park,FL
3,18995,2009,BMW,Z4,Roadster sDrive30i,95185,Gray,Black,1,5,Fleet,Englewood,CO
4,6500,2010,BMW,X3,xDrive30i AWD,126832,Red,Beige,0,3,Personal,Bountiful,UT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74590,12687,2014,Jeep,Patriot,Latitude 4WD,101479,White,Gray,0,5,Personal,Littleton,CO
74591,23409,2016,Mercedes-Benz,GLA,GLA 250 4MATIC,35602,White,Brown,0,5,Personal,Orlando,FL
74592,8191,2012,Chevrolet,Sonic,LT 2LT Sedan AT,130163,Red,Black,0,5,Personal,Ft Collins,CO
74593,16998,2014,Mazda,Mazda5,Sport Automatic,58600,Silver,Black,0,5,Personal,Oxnard,CA


### **Step 2: Get Schema Information**

In [19]:
with open("clean_car_listings_schema.txt", "r") as file:
    for line in file:
        print(line)

TRAINING DATA SCHEMA



price - price of the vehicle listed.

model_year - model year of the vehicle listed.

make - make of the vehicle listed.

trim - trim level of the vehicle specified.

mileage - number of miles on the vehicle's odometer.

exterior_color - exterior color of the vehicle listed.

interior_color - interior color of the vehicle listed.

num_accidents - number of accidents the vehicle listed has been involved in.

num_owners - number of owners associated with the vehicle listed.

usage_type - identifies whether the vehicle listed was used as part of a fleet or for personal use.

city - city where the vehicle is listed.

state - state where the vehicle is listed.


### **Step 3: View Descriptive Statistics**

In [22]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
training_data.describe()

,price,model_year,mileage,num_accidents,num_owners
count,74595.00,74595.00,74595.00,74595.00,74595.00
mean,28190.25,2018.45,52617.49,0.32,1.60
std,11695.49,3.03,33815.88,0.61,0.87
min,2499.00,2005.00,5.00,0.00,0.00
25%,19998.00,2017.00,26807.50,0.00,1.00
50%,25998.00,2019.00,46246.00,0.00,1.00
75%,33998.00,2021.00,73691.50,1.00,2.00
max,99998.00,2024.00,170000.00,7.00,9.00


##### **Key Insights:**
###### - There doesn't seem to be any null values present in any of the variables
###### - There is a lot of variablity shown in the price, mileage, num_accidents, and num_owners column (see difference betwen 75% and max).
###### - The mean for the "model_year" column is high, which could indicate bias towards newer vehicles.

### **Step 4: Univariate EDA**

##### **Model Year Bar Chart**

In [34]:
listings_by_year = sqldf("SELECT model_year, COUNT(*) AS num_listings FROM training_data GROUP BY model_year")
fig = px.bar(listings_by_year, x='model_year', y='num_listings', title='# of Listings by Model Year')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Values are skewed to the left.
###### - There is very little data to support predictions between 2005 and 2011

#### **Price Histogram**

In [39]:
fig = px.histogram(training_data, x='price', nbins=100,title='# of Listings by Price')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Values are skewed to the right.
###### - Little evidence to support values above $50,000 or below $8,000

#### **Make Bar Chart**

In [41]:
listings_by_make = sqldf("SELECT make, COUNT(*) AS num_listings FROM training_data GROUP BY make ORDER BY num_listings DESC")
fig = px.bar(listings_by_make, x='make', y='num_listings', title='# of Listings by Make')
fig.update_yaxes(showgrid=False)

##### **Key Insights**:
###### - This data seems to represent the real world: Toyota, Honda, Ford, Chevrolet are all top sellers, whereas Luxury Brands such as Cadillac, Volvo, and INFINITI sell less models.
###### - There needs to be a clear cut off point for evidence. For instance, MINI only sells 6 distinct models, whereas Cadillac sells over 30.

#### **Model Data Table**

In [47]:
listings_by_model = sqldf("SELECT make || ' ' || model AS model, COUNT(*) AS num_listings FROM training_data GROUP BY model ORDER BY num_listings DESC")



fig = go.Figure(data=[go.Table(
    header=dict(values=list(listings_by_model.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[listings_by_model.model, listings_by_model.num_listings],
               fill_color='lavender',
               align='left'))
])

fig.show()

##### **Key Insights:**
###### - Dataset seems to represent the Real World: Honda CR-V, Toyota Camry, Ford F-150 are commonly occurring models, whereas luxury or performance based vehicles are rare.
###### - There needs to be a clear cutoff point for evidence-based inference.

#### **Mileage Histogram**

In [49]:
fig = px.histogram(training_data, x='mileage', nbins=50, title='# of Listings by Mileage')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Values are skewed to the right.
###### - Little evidence to support values above 120,000 miles

#### **Exterior Color Bar Chart**

In [50]:
listings_by_ext_color = sqldf("SELECT exterior_color, COUNT(*) AS num_listings FROM training_data GROUP BY exterior_color ORDER BY num_listings DESC")
fig = px.bar(listings_by_ext_color, x='exterior_color', y='num_listings', title='# of Listings by Exterior Color')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Little evidence to support "Green", "Unknown", and "Brown"
###### - We need to establish a clear cutoff point for evidence-based inference

#### **Interior Color Bar Chart**

In [51]:
listings_by_int_color = sqldf("SELECT interior_color, COUNT(*) AS num_listings FROM training_data GROUP BY interior_color ORDER BY num_listings DESC")
fig = px.bar(listings_by_int_color, x='interior_color', y='num_listings', title='# of Listings by Interior Color')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Vast majority of values are either "Black" or "Gray"
###### - Little evidence to support values other than "Black", "Gray", or "Unknown"
###### - We need to establish a clear cutoff point for evidence-based inference

#### **# of Accidents Bar Chart**

In [53]:
listings_by_accidents = sqldf("SELECT num_accidents, COUNT(*) AS num_listings FROM training_data GROUP BY num_accidents ORDER BY num_listings DESC")
fig = px.bar(listings_by_accidents, x='num_accidents', y='num_listings', title='# of Listings by # of Accidents')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Vast majority of values are 0, 1, or 2
###### - Very little evidence to support values less than 2
###### - We need to establish a clear cutoff point for evidence-based inference

#### # of Owners Bar Chart

In [54]:
listings_by_owners = sqldf("SELECT num_owners, COUNT(*) AS num_listings FROM training_data GROUP BY num_owners ORDER BY num_listings DESC")
fig = px.bar(listings_by_owners, x='num_owners', y='num_listings', title='# of Listings by # of Owners')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Most values are either 1, 2, 3, or 4
###### - Very little evidence to support values with either 0 or 5+ owners
###### - We need to establish a clear cutoff point for evidence-based inference

#### **Usage Type Bar Chart**

In [55]:
listings_by_usage = sqldf("SELECT usage_type, COUNT(*) AS num_listings FROM training_data GROUP BY usage_type ORDER BY num_listings DESC")
fig = px.bar(listings_by_usage, x='usage_type', y='num_listings', title='# of Listings by Usage Type')
fig.update_yaxes(showgrid=False)

##### **Key Insights:**
###### - Dataset is representative of the Real World, where the majority of vehicles sold are used for personal use cases rather than fleets.
###### - Although the majority of the values support "Personal" use cases, there is enough data to support "Fleet" use cases.

### **Step 5: Multivariate EDA**

In [58]:
numeric_features = training_data[['price', 'model_year', 'mileage', 'num_accidents', 'num_owners']]
numeric_features.corr()

,price,model_year,mileage,num_accidents,num_owners
price,1.00,0.57,-0.57,-0.16,-0.31
model_year,0.57,1.00,-0.74,-0.19,-0.51
mileage,-0.57,-0.74,1.00,0.19,0.44
num_accidents,-0.16,-0.19,0.19,1.00,0.13
num_owners,-0.31,-0.51,0.44,0.13,1.00


##### **Key Insights:**
###### - There is a moderately strong correlation between price and model year and price and mileage.
###### - All numeric variables, excluding num_accidents, have at least slightly strong correlation with the target variable